#FlipFeed

###Objetivo:
> Identificar cuando una persona realiza una dominada correctamente

###Metodo:
> Usaremos en este caso, redes neuronales recurrentes para obtener un valor booleano

###Herramientas:

0.   Python
1.   Keras - TensorFLow
2.   Yolo v8 - entrenado con un data set personalizado
3.   OpenCV optical flow






####Montando el directorio del drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
global dirFlipfeed, dataSetPath, dirTrV, dirOut
dirFlipfeed = '/content/drive/Shareddrives/flipfeed'
dataSetPath  =  '/content/drive/Shareddrives/flipfeed/dataSetToTraining/ballAndPerson'
dirTrV = './drive/Shareddrives/flipfeed/TrainingVideos'
dirOut = './drive/Shareddrives/flipfeed/Output'

##Yolo v8 y OpenCV Optical Flow
Yolov8 lo utilizamos para etiquetar los objetos en el video. Lo entrenamos con un data set custom, podiendo así extraer las coordenadas en donde se encuentra la persona y el balón.

#### 👇 Instalamos todos los paquetes necesarios.

In [2]:
!pip install torch numpy opencv-python ultralytics supervision==0.1.0 

In [3]:
# Import the necesary libraries
import torch
import numpy as np
import cv2
from time import time
from ultralytics import YOLO

from supervision.draw.color import ColorPalette
from supervision.tools.detections import Detections, BoxAnnotator
from IPython import display

WARNING  Ultralytics settings reset to defaults. This is normal and may be due to a recent ultralytics package update, but may have overwritten previous settings. 
View and update settings with 'yolo settings' or at 'C:\Users\Striwey\AppData\Roaming\Ultralytics\settings.yaml'


 #### 👉 Entrenando Yolo v8 small con un data set custom


In [5]:
!yolo task=detect mode=train model=yolov8s.pt data={dataSetPath}/data.yaml epochs=25 imgsz=1599 plots=True batch=-1
# Data set custom extracted from (url)

100% 21.5M/21.5M [00:00<00:00, 32.8MB/s]
Ultralytics YOLOv8.0.112 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/Shareddrives/flipfeed/dataSetToTraining/ballAndPerson/data.yaml, epochs=25, patience=50, batch=-1, imgsz=1599, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina

#### 👍 Validando el modelo entrenado

In [8]:
!yolo task=detect mode=val model={dirFlipfeed}/runs/detect/train/weights/best.pt data={dataSetPath}/data.yaml

Ultralytics YOLOv8.0.112 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients
val: Scanning /content/drive/Shareddrives/flipfeed/dataSetToTraining/ballAndPerson/valid/labels.cache... 320 images, 0 backgrounds, 0 corrupt: 100% 320/320 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 20/20 [00:40<00:00,  2.05s/it]
                   all        320       1103      0.561      0.426      0.446      0.246
                  Ball        320        304      0.609      0.559      0.563      0.353
                Person        320        799      0.513      0.293      0.329       0.14
Speed: 4.5ms preprocess, 50.2ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/val2


In [ ]:
#To inference
!yolo task=detect mode=predict model={dirFlipfeed}/runs/detect/train/weights/best.pt conf=0.25 source={dataSetPath}/test/images save=True

In [5]:
def predict(self, frame):
       
    model = YOLO('./runs/detect/train/weights/best.pt')
    results = model.predict(frame)
    
    return results
    

def plot_bboxes(self, results, frame):

    xyxys = []
    confidences = []
    class_ids = []

    # Extract detections for person class
    for result in results[0]:
        class_id = result.boxes.cls.cpu().numpy().astype(int)

        if class_id == 0:

            xyxys.append(result.boxes.xyxy.cpu().numpy())
            confidences.append(result.boxes.conf.cpu().numpy())
            class_ids.append(result.boxes.cls.cpu().numpy().astype(int))


    # Setup detections for visualization
    detections = Detections(
                xyxy=results[0].boxes.xyxy.cpu().numpy(),
                confidence=results[0].boxes.conf.cpu().numpy(),
                class_id=results[0].boxes.cls.cpu().numpy().astype(int),
                )


    # Format custom labels
    labels = [f"{self.CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
    for _, confidence, class_id, tracker_id
    in detections]

    # Annotate and display frame
    frame = self.box_annotator.annotate(frame=frame, detections=detections, labels=labels)

    return frame


cap = cv2.VideoCapture(0)
assert cap.isOpened()
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while True:
          
    start_time = time()

    ret, frame = cap.read()
    assert ret

    results = predict(frame)
    frame = self.plot_bboxes(results, frame)

    end_time = time()
    fps = 1/np.round(end_time - start_time, 2)

    cv2.putText(frame, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)

    cv2.imshow('YOLOv8 Detection', frame)

    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

TypeError: predict() missing 1 required positional argument: 'frame'

##OpenCV Optical Flow

In [ ]:
#To draw the lines on the frame
def draw_flow(img, flow, step=20):

    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T

    lines = np.vstack([x, y, x-fx, y-fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(img_bgr, lines, 0, (0, 255, 0))

    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(img_bgr, (x1, y1), 1, (0, 255, 0), -1)

    return img_bgr

#Capture the video and generate a output with cv2
videoPath = 'dom2.mp4'
#a = open(dirTrV + videoPath, 'r')
input = cv2.VideoCapture(dirTrV + '/'+videoPath)
output = cv2.VideoWriter(dirOut + '/output_' + videoPath, cv2.VideoWriter_fourcc(*'mp4v'), 10, (int(input.get(3)),int(input.get(4))))

suc, prev = input.read()
prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

while (input.isOpened()):
    #Extract a frame from video and put on it a B&W format 
    suc, img = input.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #Obtain the motion prediction
    flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    prevgray = gray

    #Write frames on output
    imgToShow = draw_flow(gray, flow)
    output.write(imgToShow)

output.release()
input.release()
cv2.destroyAllWindows()

##RNN implement with Keras - Tensorflow

In [ ]:
import tensorflow as tf
import tensorflow.keras as kr
from tensorflow.keras import layers

lr = 0.01           # learning rate
nn = [2, 16, 8, 1]  # número de neuronas por capa.

model = kr.Sequential()

model.add(layers.Embedding(input_dim=1000, output_dim=64))
model.add(layers.SimpleRNN(128))
model.add(layers.Dense(10))

model.summary()

# Compilamos el modelo, definiendo la función de coste y el optimizador.
#model.compile(loss='mse', optimizer=kr.optimizers.SGD(lr=0.05), metrics=['acc'])

# Y entrenamos al modelo. Los callbacks 
#model.fit(X, Y, epochs=100)